In [41]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import bokeh as bk
from bokeh.io import output_notebook
import seaborn as sns

from compound import (
    get_compound_return, 
    simulate_compound_return,
    build_dataframe,
    MONTHS_IN_YEAR,
    DEFAULT_RETIREMENT_YEARS,
    define_scenario
)

output_notebook()

Loading BokehJS ...

In [171]:
import inspect
class Mortage():
    def __init__(
        self, 
        principal_amount,
        annual_interest_rate,
        duration_years,
        currency="€"
    ):
        self.principal_amount = principal_amount
        self.annual_interest_rate = annual_interest_rate / 100
        self.monthly_interest_rate = annual_interest_rate / 12 / 100
        self.duration_years = duration_years
        self.total_payments = duration_years * 12
        self.currency = currency

    def __getattr__(self, name):
        value = getattr(self, f"_{name}", None)
        if isinstance(value, float):
            return round(value, 2)
        else:
            return value
    
    def set_down_payment(self, down_payment):
        if down_payment >= self.principal_amount:
            self.down_payment = self.principal_amount
            print("The current down payment is sufficient to amortice the mortage")
        else:
            self.down_payment = down_payment
            self.remaining_principal = self.principal_amount - self.down_payment
            self.installment = self.remaining_principal * self.monthly_interest_rate / (1 - (1 + self.monthly_interest_rate) ** -self.total_payments)
            self.total_to_amortice = down_payment + self.total_payments * self.installment
    def amortization_plan(self, verbose=True):
        if self.down_payment >= self.principal_amount:
            print("Down payment has amorticed the mortage")
            return []
        
        amortization_schedule = []
        self.principals = []
        self.interests = []
        self.terms = []

        for month in range(1, self.total_payments + 1):
            interest_payment = self.remaining_principal * self.monthly_interest_rate
            principal_payment = self.installment - interest_payment
            self.remaining_principal -= principal_payment
            
            self.terms.append(month)
            self.principals.append(round(principal_payment, 1))
            self.interests.append(round(interest_payment, 1))
            amortization_schedule.append((month, round(self.installment, 2), round(principal_payment, 2), round(interest_payment, 2), round(self.remaining_principal, 2)))
        self.total_interests = sum(self.interests)
        self.total_principals = sum(self.principals)
        
        if verbose:
            print(f"Total interest {round(self.total_interests, 2)}")
            print(f"Total principal {round(self.total_principals, 3)}")
            print(f"Total amount (down payment + interest + principal) \n{round(self.total_to_amortice, 2)}")

        
        return amortization_schedule

    def plot_schedule(self, w=400, h=250, l_w=2):
        source = bk.models.ColumnDataSource(data={"terms": self.terms, 
                                                  "principals": self.principals, 
                                                  "interests": self.interests
                                         }
                                   )
        hover = bk.models.HoverTool(tooltips=[("Term", "@terms"), 
                                              ("Installment", f"{self.currency} {self.installment}"),
                                              ("Principal", f"{self.currency} @principals"),
                                              ("Interest", f"{self.currency} @interests")
                                             ]
                                   )
        plot = bk.plotting.figure(plot_height=h, 
                                  plot_width=w,
                                  tools=[hover]
                                 )
        # x-axis
        plot.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
        plot.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
        # y-axis
        plot.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
        plot.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
        
        plot.line("terms", 
                  "principals", 
                  source=source, 
                  line_width=l_w,
                  color="cyan",
                  legend_label="principal"
                 )
        plot.line("terms", 
                  "interests", 
                  source=source, 
                  line_width=l_w,
                  color="yellow",
                  legend_label="interest"
                 )
        
        # legend
        plot.legend.label_text_font_size = '8pt'
        plot.legend.location = "center_right"
        
        bk.plotting.show(plot)

In [182]:
# Taxes and expenses (the value is an estimate from Spanish IVA (10%) and AJD (0.75))
# It might be less.
taxes_plus_expenses = (0.1 + 0.0075) * 200_000
m = Mortage(200_000 + taxes_plus_expenses, 4.0, 40)
m.set_down_payment(190_000)
schedule = m.amortization_plan()

m.plot_schedule(600, 300)

Total interest 31692.9
Total principal 31499.1
Total amount (down payment + interest + principal) 
253192.3
